# Analiza Wielowymiarowa - zajecia 8 - Hierarchiczna analiza skupień

In [ ]:
import os
os.getcwd() # oczekiwany .../AWXXXX/materialy/zajecia08
# mozna uzyc os.chdir("path") do zmiany

In [ ]:
import yaml
spec =  yaml.safe_load(open('../../spec.yaml'))

In [ ]:
# STATA
import stata_setup
stata_setup.config(spec["stata_path"], spec["stata_type"])
from pystata import stata

In [ ]:
import pandas as pd
import scipy
import sklearn
import numpy as np
import pyAesCrypt
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    
# Hierarchical
# https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering

### Przykład 1.

Dane i przykład zostały pożyczone z podręcznika  
Sophia Rabe-Hesketh i Brian Everitt  
"A Handobook of Statistical Analyses using Stata"  
Dane dotyczą czaszek ludzkich znalezionych w Tybecie

In [ ]:
if not os.path.isfile("../../dane/tibetan.dta"):
    password = spec["password_pyaescrypt"]
    if password is None:
        password = input("password: ")
    pyAesCrypt.decryptFile("../../dane/tibetan.dta.aes", "../../dane/tibetan.dta", password)

In [ ]:
%stata use ../../dane/tibetan.dta, clear  

In [ ]:
tibetan = pd.read_stata("../../dane/tibetan.dta")

In [ ]:
%%stata
/// opis danych
des
sum

In [ ]:
tibetan.describe().T

In [ ]:
# %stata help cluster

In [ ]:
%%stata
/// Zmienne sa mierzone w identycznych jednostkach (mm)
/// Nie ma potrzeby standaryzacji wartosci zmiennych
/// Metoda pojedynczego wiazania
cluster singlelinkage length breadth height upper face, name(pojedyncze)
cluster dendrogram

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "single")
model = model_base.fit(tibetan[["length", "breadth", "height", "upper", "face"]])
plt.title("Hierarchical Clustering Dendrogram - single")
# Default dendogram, looks quite similar to default STATA
plot_dendrogram(model)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
%%stata
/// Metoda pelnego wiazania
cluster completelinkage length breadth height upper face, name(pelne)
cluster dendrogram

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "complete")
model = model_base.fit(tibetan[["length", "breadth", "height", "upper", "face"]])
plt.title("Hierarchical Clustering Dendrogram - complete")
# Default dendogram, looks quite similar to default STATA
plot_dendrogram(model)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

ten dendrogram wskazuje na hierarchiczny charakter danych
dlugosc pionowych linii pokazuje roznice miedzy skupieniami
im linie sa dluzsze tym obiekty bardziej roznia sie

Metoda przecietnego wiazania

In [ ]:
%%stata
cluster averagelinkage length breadth height upper face, name(przecietne)
cluster dendrogram

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "average")
model = model_base.fit(tibetan[["length", "breadth", "height", "upper", "face"]])
plt.title("Hierarchical Clustering Dendrogram - average")
# Default dendogram, looks quite similar to default STATA
plot_dendrogram(model)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
%%stata
/// Metoda medianowego wiazania
cluster medianlinkage length breadth height upper face, name(medianowe)

In [ ]:
%stata cluster list

In [ ]:
%%stata
/// Metoda Warda
cluster wardslinkage length breadth height upper face, name(Warda)
cluster dendrogram
/// ten dendrogram wskazuje na hierarchiczny charakter danych
/// prosze zwrocic uwage ze metoda Warda wykorzystuje inna miare odleglosci

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "ward")
model = model_base.fit(tibetan[["length", "breadth", "height", "upper", "face"]])
plt.title("Hierarchical Clustering Dendrogram - ward")
# Default dendogram, looks quite similar to default STATA
plot_dendrogram(model)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
%%stata
/// Kryterium wyboru optymalnej liczby grup
cluster stop pojedyncze, rule(duda)
cluster stop pojedyncze, rule(calinski)
cluster dendrogram pojedyncze 
cluster dendrogram pojedyncze, cutn(6) showcount

In [ ]:
%%stata
cluster stop pelne, rule(duda)
cluster stop pelne, rule(calinski)
cluster dendrogram pelne 
cluster dendrogram pelne, cutn(3) showcount

In [ ]:
%%stata
cluster stop przecietne, rule(duda)
cluster stop przecietne, rule(calinski)

cluster stop medianowe, rule(duda)
cluster stop medianowe, rule(calinski)

cluster stop Warda, rule(duda)
cluster stop Warda, rule(calinski)

In [ ]:
%%stata
/// Chcemy zobaczyc charakterystyki grup czaszek uzyskanych metoda pelnego wiazania
/// Tworzymy identyfikatory grup
cluster generate grupa = groups(3), name(pelne)

In [ ]:
%%stata
/// Tabela liczebnosci
tabulate grupa

### Przykład 2

W tym przykladzie wykorzystamy znane juz Panstwu dane dotyczace trzech odmian irysow
Ich cechy to:  
- Dlugosc platka [cm] (petal lenght)
- Szerokosc platka [cm] (petal width)
- Dlugosc listka kielicha [cm]
- Szerokosc listka kielicha [cm]


In [ ]:
%stata use ../../dane/iris.dta, clear

In [ ]:
iris = pd.read_stata("../../dane/iris.dta")
iris_x = iris.iloc[:,1:]
iris_y = iris.iloc[:,0]

In [ ]:
%stata des

In [ ]:
%%stata
/// Podstawowe statystyki opisowe
bysort iris: su seplen sepwid petlen petwid 

In [ ]:
iris_x.groupby(iris_y).describe().T

In [ ]:
%%stata
cluster singlelinkage seplen sepwid petlen petwid , name(pojedyncze)

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "single")
model = model_base.fit(iris_x)

In [ ]:
# from scipy.cluster.hierarchy import dendrogram
# metody obcinania wykresu, truncate_modestr 'lastp' i 'level'

Dla zbioru o wiekszej liczbie obserwacji nie mozna wyswietlic pelnego dendrogramu  
przydatne sa opcje:

1. cutnumber()           wyswietla okreslona liczbe galezi  
2. cutvalue()            wyswietla tylko rozniace sie galezie o zadana wartosc  

opcja showcount pokazuje liczbe obserwacji w kazdej galezi

In [ ]:
%stata cluster dendrogram, cutnumber(10) showcount

In [ ]:
%stata cluster dendrogram, cutvalue(0.8) showcount

In [ ]:
%%stata
/// tworzymy identyfikator 3 grup
cluster generate gr_single = groups(3), name(pojedyncze)
/// Czy klasyfikacja jest poprawna?
tab iris gr_single

In [ ]:
%%stata
/// Sprobujmy inna metoda
cluster completelinkage seplen sepwid petlen petwid , name(pelne)
cluster dendrogram, cutnumber(10)
cluster dendrogram, cutvalue(0.5)

In [ ]:
%%stata
/// tworzymy identyfikator 3 grup
cluster generate gr_complete = groups(3), name(pelne)
/// Czy klasyfikacja jest poprawna?
tab iris gr_complete

In [ ]:
%%stata
/// Sprobujmy inna metoda
cluster wardslinkage seplen sepwid petlen petwid , name(ward)
cluster dendrogram, cutnumber(10)
cluster dendrogram, cutvalue(0.5)
/// tworzymy identyfikator 3 grup
cluster generate gr_ward = groups(3), name(ward)
/// Czy klasyfikacja jest poprawna?
tab iris gr_ward